In [1]:
import pickle
import os
import tarfile
import urllib
import urllib2
from os.path import isfile, isdir
from tqdm import tqdm

import os
#import cv2
import h5py
import math
import numpy as np
from PIL import Image
from scipy import misc
import random as rand
from operator import add
import cPickle as pickle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches
import tarfile
import urllib
from IPython.display import display, Image
from scipy import ndimage
import cPickle as pickle
import scipy.io as sio
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import sys
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split
import glob
import csv

from bs4 import BeautifulSoup
import re

import nltk
from nltk.corpus import stopwords
import nltk.data

from gensim.models import word2vec

# Load model and data

In [2]:
data_root = 'data/'
try:
    os.mkdir(data_root)
except OSError:
    pass

model_root = 'model/'
try:
    os.mkdir(model_root)
except OSError:
    pass

In [3]:
with open(os.path.join(data_root,'train_x_clean.pkl'), 'rb') as f:
    train_x_clean = pickle.load(f)
    
with open(os.path.join(data_root,'train_y_clean.pkl'), 'rb') as f:
    train_y_clean = pickle.load(f)
    
with open(os.path.join(data_root,'validation_x_clean.pkl'), 'rb') as f:
    validation_x_clean = pickle.load(f)
    
with open(os.path.join(data_root,'validation_y_clean.pkl'), 'rb') as f:
    validation_y_clean = pickle.load(f)

with open(os.path.join(data_root,'test_x_clean.pkl'), 'rb') as f:
    test_x_clean = pickle.load(f)
    
with open(os.path.join(data_root,'test_y_clean.pkl'), 'rb') as f:
    test_y_clean = pickle.load(f)

In [15]:
with open(os.path.join(model_root, 'model_w2v1.pkl'), 'rb') as f:
    model_w2v1 = pickle.load(f)

with open(os.path.join(model_root, 'model_w2v2.pkl'), 'rb') as f:
    model_w2v2 = pickle.load(f)

with open(os.path.join(model_root, 'model_w2v3.pkl'), 'rb') as f:
    model_w2v3 = pickle.load(f)

with open(os.path.join(model_root, 'model_w2v4.pkl'), 'rb') as f:
    model_w2v4 = pickle.load(f)

In [7]:
#testing
model_w2v1.most_similar("awful")

[(u'terrible', 0.7557451725006104),
 (u'atrocious', 0.7374610900878906),
 (u'dreadful', 0.732129693031311),
 (u'horrible', 0.7258363366127014),
 (u'abysmal', 0.7127856016159058),
 (u'horrendous', 0.664879560470581),
 (u'appalling', 0.6576082110404968),
 (u'lousy', 0.6411954164505005),
 (u'amateurish', 0.6183844804763794),
 (u'horrid', 0.6154941320419312)]

In [8]:
len(model_w2v1.wv.index2word)

16490

# Averaging the words in the review

In [14]:
num_features1 = 300

In [9]:
#testing case
review_cleaned = train_x_clean[1000]
print review_cleaned

[u'first', u'half', u'version', u'best', u'seen', u'think', u'seen', u'every', u'version', u'jane', u'eyre', u'ever', u'made', u'development', u'jane', u'childhood', u'character', u'exceptional', u'though', u'someone', u'said', u'uh', u'oh', u'running', u'long', u'hacked', u'rest', u'story', u'shreds', u'major', u'scenes', u'included', u'glossed', u'combined', u'put', u'order', u'way', u'completely', u'change', u'storyline', u'little', u'transition', u'even', u'scene', u'development', u'would', u'difficult', u'anyone', u'familiar', u'story', u'even', u'follow', u'big', u'disappointment', u'beginning', u'opened', u'much', u'hope', u'end', u'dashed']


In [5]:
def review_average(words, model, num_features):
    """"""
    
    featureVec = np.zeros((num_features,),dtype="float32")
    vocabulary = set(model.wv.index2word)
    number_of_words = 0
    
    for idx, word in enumerate(words):
        #print word
        if word in vocabulary:
            number_of_words += 1
            featureVec = np.add(featureVec, model[word])
            
    if number_of_words != 0:    
        return np.divide(featureVec,number_of_words)
    else:
        return featureVec

In [27]:
averaged_review = review_average(review_cleaned, model_w2v1, num_features1)
print averaged_review.shape
print averaged_review

(300,)
[-0.32485157  0.15975218  0.71066684  0.1311163   0.56365752  0.12271948
  0.06935742 -0.10353193 -0.40285987 -0.00250576  0.35182038 -0.21860968
  0.40353233 -0.18843099  0.06319015 -0.00174407 -0.2742798  -0.04080088
 -0.17614974 -0.1298345   0.36167276 -0.26331189  0.16672044  0.11894622
 -0.09420048  0.49859056 -0.17302056  0.03466947  0.09091568 -0.09192493
  0.23952366 -0.20294222  0.26644468 -0.67126548  0.13869768 -0.00264744
  0.12139206  0.2349679   0.20816085  0.03638868  0.53511643  0.15586545
  0.07965149  0.55755979 -0.3297694  -0.29596037 -0.05610412 -0.03870651
 -0.16199851 -0.2199301  -0.05139443 -0.03169621  0.38437524  0.38536841
  0.18398978  0.12839681  0.00760228  0.28749058  0.18202116 -0.18075319
 -0.23738824 -0.32781109 -0.86737877  0.11596496 -0.5699228  -0.25697047
  0.03668885 -0.15477724  0.02013704  0.02843265 -0.09299009 -0.10662208
 -0.23372228  0.19216651  0.04211576  0.28296244  0.28956273  0.02974446
 -0.01731472 -0.2605896   0.00574272  0.1941

In [6]:
def dataset_average(dataset, model, num_features):
    new_dataset = []
    for review in dataset:
        review_averaged = review_average(review, model, num_features)
        new_dataset.append(review_averaged)
    return new_dataset

In [38]:
train_x_average = dataset_average(train_x_clean, model_w2v1, 300)
validation_x_average = dataset_average(validation_x_clean, model_w2v1, 300)
test_x_average = dataset_average(test_x_clean, model_w2v1, 300)

In [35]:
print len(train_x_average)
print len(validation_x_average)
print len(test_x_average)
print train_x_average[1000].shape
print validation_x_average[1000].shape
print test_x_average[1000].shape

25000
12500
12500
(300,)
(300,)
(300,)


In [41]:
files = {'train_x_average': train_x_average,
         'validation_x_average': validation_x_average,
         'test_x_average': test_x_average}

for filename, afile in files.items():
    
    file_path = os.path.join(data_root,filename+'.pkl')
    
    if not isfile(file_path):    
        with open(file_path, 'wb') as f:
            pickle.dump(afile, f)

# Train classiciation models

In [39]:
from sklearn.ensemble import RandomForestClassifier
rd_clf = RandomForestClassifier(n_estimators = 100, random_state=3)
rd_clf = rd_clf.fit(train_x_average, train_y_clean)
accuracy_validation = rd_clf.score(validation_x_average, validation_y_clean)
#test_validation = rd_clf.score(test_features_bw, test_labels)
print accuracy_validation

0.82312


In [40]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf = lr_clf.fit(train_x_average, train_y_clean)
accuracy_validation = lr_clf.score(validation_x_average, validation_y_clean)
#test_validation = lr_clf.score(test_reviews_average, test_labels)
print accuracy_validation

0.8732


In [41]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC()
svm_clf = svm_clf.fit(train_x_average, train_y_clean)
accuracy_validation = svm_clf.score(validation_x_average, validation_y_clean)
#test_validation = svm_clf.score(test_re_average, test_labels)
print accuracy_validation

0.87384


# Model_w2v2

In [9]:
train_x_average2 = dataset_average(train_x_clean, model_w2v2, 600)
validation_x_average2 = dataset_average(validation_x_clean, model_w2v2, 600)
test_x_average2 = dataset_average(test_x_clean, model_w2v2, 600)

In [24]:
from sklearn.ensemble import RandomForestClassifier
rd_clf2 = RandomForestClassifier(n_estimators = 100, random_state=3)
rd_clf2 = rd_clf2.fit(train_x_average2, train_y_clean)
accuracy_validation = rd_clf2.score(validation_x_average2, validation_y_clean)

print accuracy_validation

0.83544


In [25]:
from sklearn.linear_model import LogisticRegression
lr_clf2 = LogisticRegression()
lr_clf2 = lr_clf2.fit(train_x_average2, train_y_clean)
accuracy_validation = lr_clf2.score(validation_x_average2, validation_y_clean)
#test_validation = lr_clf.score(test_reviews_average, test_labels)
print accuracy_validation

0.87392


In [27]:
from sklearn.svm import LinearSVC
svm_clf2 = LinearSVC()
svm_clf2 = svm_clf2.fit(train_x_average2, train_y_clean)
accuracy_validation = svm_clf2.score(validation_x_average2, validation_y_clean)
#test_validation = svm_clf.score(test_re_average, test_labels)
print accuracy_validation

0.87592


# w2v model3

In [11]:
train_x_average3 = dataset_average(train_x_clean, model_w2v3, 300)
validation_x_average3 = dataset_average(validation_x_clean, model_w2v3, 300)
test_x_average3 = dataset_average(test_x_clean, model_w2v3, 300)

In [29]:
from sklearn.ensemble import RandomForestClassifier
rd_clf3 = RandomForestClassifier(n_estimators = 100, random_state=3)
rd_clf3 = rd_clf3.fit(train_x_average3, train_y_clean)
accuracy_validation = rd_clf3.score(validation_x_average3, validation_y_clean)

print accuracy_validation

0.77024


In [30]:
from sklearn.linear_model import LogisticRegression
lr_clf3 = LogisticRegression()
lr_clf3 = lr_clf3.fit(train_x_average3, train_y_clean)
accuracy_validation = lr_clf3.score(validation_x_average3, validation_y_clean)

print accuracy_validation

0.80968


In [31]:
from sklearn.svm import LinearSVC
svm_clf3 = LinearSVC()
svm_clf3 = svm_clf3.fit(train_x_average3, train_y_clean)
accuracy_validation = svm_clf3.score(validation_x_average3, validation_y_clean)
#test_validation = svm_clf.score(test_re_average, test_labels)
print accuracy_validation

0.83784


# W2V model4

In [16]:
train_x_average4 = dataset_average(train_x_clean, model_w2v4, 100)
validation_x_average4 = dataset_average(validation_x_clean, model_w2v4, 100)
test_x_average4 = dataset_average(test_x_clean, model_w2v4, 100)

In [33]:
from sklearn.ensemble import RandomForestClassifier
rd_clf4 = RandomForestClassifier(n_estimators = 100, random_state=3)
rd_clf4 = rd_clf4.fit(train_x_average4, train_y_clean)
accuracy_validation = rd_clf4.score(validation_x_average4, validation_y_clean)
#test_validation = rd_clf.score(test_features_bw, test_labels)
print accuracy_validation

0.8156


In [34]:
from sklearn.linear_model import LogisticRegression
lr_clf4 = LogisticRegression()
lr_clf4 = lr_clf4.fit(train_x_average4, train_y_clean)
accuracy_validation = lr_clf4.score(validation_x_average4, validation_y_clean)
#test_validation = lr_clf.score(test_reviews_average, test_labels)
print accuracy_validation

0.84584


In [35]:
from sklearn.svm import LinearSVC
svm_clf4 = LinearSVC()
svm_clf4 = svm_clf4.fit(train_x_average4, train_y_clean)
accuracy_validation = svm_clf4.score(validation_x_average4, validation_y_clean)
#test_validation = svm_clf.score(test_re_average, test_labels)
print accuracy_validation

0.84616


In [46]:
print "Accuracy on the test set"
print "w2v1: default structure and algorith, dowmsampling size = 1e-3, num_features = 300"
print "random forest: ", rd_clf.score(test_x_average, test_y_clean)
print "logisic regression: ", lr_clf.score(test_x_average, test_y_clean)
print "svr: ", svm_clf.score(test_x_average, test_y_clean)
print "\n"

print "w2v2: default structure and algorith, dowmsampling size = 1e-3, num_features = 600"
print "random forest: ", rd_clf2.score(test_x_average2, test_y_clean)
print "logisic regression: ", lr_clf2.score(test_x_average2, test_y_clean)
print "svm: ", svm_clf2.score(test_x_average2, test_y_clean)
print "\n"

print "w2v3: default structure and algorith, dowmsampling size = 1e-5, num_features = 300"
print "random forest: ", rd_clf3.score(test_x_average3, test_y_clean)
print "logisic regression: ", lr_clf3.score(test_x_average3, test_y_clean)
print "svm: ", svm_clf3.score(test_x_average3, test_y_clean)
print "\n"

print "w2v4: negative sampling, dowmsampling size = 1e-3m, num_features = 100"
print "random forest: ", rd_clf4.score(test_x_average4, test_y_clean)
print "logisic regression: ", lr_clf4.score(test_x_average4, test_y_clean)
print "svm:", svm_clf4.score(test_x_average4, test_y_clean)

Accuracy on the test set
w2v1: default structure and algorith, dowmsampling size = 1e-3, num_features = 300
random forest:  0.8296
logisic regression:  0.87176
svr:  0.8724


w2v2: default structure and algorith, dowmsampling size = 1e-3, num_features = 600
random forest:  0.83176
logisic regression:  0.87264
svm:  0.87288


w2v3: default structure and algorith, dowmsampling size = 1e-5, num_features = 300
random forest:  0.77256
logisic regression:  0.81056
svm:  0.8368


w2v4: negative sampling, dowmsampling size = 1e-3m, num_features = 100
random forest:  0.814
logisic regression:  0.84464
svm: 0.84424
